### 0. Initialisation

In [ ]:
#Chargement des packages
import numpy as np 
import pandas as pd 
import warnings
import datetime as dt
import pytz #to handle timezones

from google.cloud import bigquery
from google.oauth2 import service_account

#Chemins
PATH_output = ""

#Options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = "{:.2f}".format
warnings.filterwarnings('ignore')


### 1. Connexion Big Query

In [465]:
client = bigquery.Client(project='transformed-data')
project = client.project


### 2. Charger les données de Big Query

In [499]:
# Charger les données risque des contrats souscrits

query = """
    select policy_number, period_status, ph_email, policy_risk_profile, md_crm_coefficient, sd_driver_id, vh_financing_method, online_offline
    from mart_insurance.mart_submissions as s
    where period_status = "Bound"
    """
query_job = client.query(query)
acc_bound = query_job.to_dataframe()
acc_bound.shape

(40487, 165)

In [ ]:
# Charger les données des documents

query = """
    select *
    from mart_insurance.mart_documents as s
    """
query_job = client.query(query)
doc = query_job.to_dataframe()
doc.shape

In [ ]:
# Charger les données des polices validées

query = """
    select policy_number
    from mart_insurance.mart_bound_policies
    where issue_date is not null
    """
query_job = client.query(query)
val = query_job.to_dataframe()
val.shape

### 3. Mapper les infos risque et de validation à la base des documents

In [ ]:
# Mapper les infos des contrats : email, risk profil, CRM, conducteur secondaire, mode d'achat du véhicule

doc2 = doc.merge(acc_bound, on = "policy_number", how = "inner").merge(val, on = "policy_number", how = "left", indicator = True)
doc2['policy_issued'] = False
doc2.loc[doc2._merge == "both", 'policy_issued'] = True

### 4. Pre-processing base des documents

In [ ]:
# Garder les variables pertinentes

doc3 = doc2[[
    'policy_number',
    'policy_issued',
    'period_start',
    'is_mandatory',
    'ph_email', 
    'online_offline',
    'document_type', 
    'document_status',
    'document_sent_at',
    'document_received_at',
    'document_validated_at',
    'comments',
    'is_validated',
    'policy_risk_profile',
    'sd_driver_id',
    'md_crm_coefficient',
    'vh_financing_method'
]]

# Pré-processing

doc3['md_crm_coefficient'] = doc3.md_crm_coefficient.astype(float)
for col in doc3.columns:
   if doc3[col].dtype in ("datetime64[ns, UTC]", "datetime64[ns]", "dbdate") :
      doc3[col] = pd.to_datetime(doc3[col]).dt.date    

# Délai avant résiliation

doc3['supposed_cancel_date'] = pd.to_datetime((doc3['period_start'] + timedelta(days = 55)))
doc3['delay_before_cancel'] = ((doc3['supposed_cancel_date'] - pd.to_datetime(dt.datetime.now().date())) / np.timedelta64(1, 'D')).astype(int)


### 5. Docusign

In [ ]:
# Importer les données Docusign

ds = pd.read_csv("../../input_docusign_20230608.csv', low_memory = False)

# Liste des documents signés dédoublonnés par email

ds_nodup = ds.loc[ds['État'] == "Complété"].drop_duplicates('Adresse e-mail du destinataire')[['Adresse e-mail du destinataire']]
ds_nodup.rename(columns = {'Adresse e-mail du destinataire' : 'ph_email'}, inplace = True)

# Mapper l'info à la DB des documents par email

doc4 = doc3.merge(ds_nodup, on = "ph_email", how = "left", indicator = True)
doc4['Found_Docusign'] = False
doc4.loc[doc4._merge == "both", 'Found_Docusign'] = True
doc4.drop(['_merge'], axis = 1, inplace = True)

In [ ]:
# Recalculer la validation en prenant en compte l'info Docusign

doc4['is_validated_Clem'] = ((doc4.document_status == "Validé") | ((doc4.document_type == "Contrat signe") & (doc4.Found_Docusign == True))) 

### 6. Règles pour l'obligation des documents

In [ ]:
# Recalculer le caractère obligatoire des documents

doc4['is_mandatory_Clem'] = False

#Relevé d'information : obligatoire pour tous sauf (les Sans Ant. & Jeunes) avec CRM = 1
doc4.loc[(doc4.document_type == "Relevé d'information des différentes compagnies d'assurance"), 'is_mandatory_Clem'] = True
doc4.loc[((doc4.policy_risk_profile.isin(['No Previous Insurance', 'Young Driver'])) & (doc4.md_crm_coefficient == 1) & (doc4.document_type == "Relevé d'information des différentes compagnies d'assurance")), 'is_mandatory_Clem'] = False

# Permis & Certificat d’immatriculation & Contrat signé : obligatoire pour tous
doc4.loc[doc4.document_type.isin(['Permis de conduire', 'Certificat d’immatriculation', 'Contrat signe']), 'is_mandatory_Clem'] = True

# Permis du conducteur secondaire : obligatoire pour ceux qui ont un conducteur secondaire
doc4.loc[(~(doc4.sd_driver_id.isna()) & (doc4.document_type == "Permis de conduire un second conducteur")), 'is_mandatory_Clem'] = True

# Contrat de location : obligatoire pour ceux qui ont un mode de financement Leasing with purchase (LOA) ou Leasing (LLD)
#doc4.loc[(doc4.document_type == "Contrat de location") & (doc4.vh_financing_method.isin(['Leasing with purchase', 'Leasing'])), 'is_mandatory_Clem'] = True

In [ ]:
# Calculer à la maille police le nb de documents obligatoires vs nb de documents validés

doc4['is_pending_Clem'] = ((doc4.is_mandatory_Clem == True) & (doc4.is_validated_Clem == False))

list_col = ['is_mandatory_Clem', 'is_validated_Clem', 'is_pending_Clem']
temp = doc4[(list(['policy_number'] + list_col))]

for col in list_col:
   temp[col] = temp[col].astype(int)    

temp = temp.groupby(['policy_number'])[['is_mandatory_Clem', 'is_validated_Clem', 'is_pending_Clem']].sum().reset_index()
temp['is_pending_policy_Clem'] = 0
temp.loc[temp.is_pending_Clem > 0, 'is_pending_policy_Clem'] = 1

temp.rename(columns = {'is_pending_Clem' : 'nb_documents_pending_Clem'}, inplace = True)

In [ ]:
# Mapper à la maille police s'il y a des documents pending

doc5 = doc4.merge(temp[['policy_number', 'is_pending_policy_Clem', 'nb_documents_pending_Clem']], on = "policy_number", how = "left")

### 7. Export final


In [ ]:
file_path = PATH_output + '/Stats Documents'
doc_sql5.to_csv(file_path + '.csv', encoding = 'utf-8-sig')